In [51]:
#导入dateframe计算工具
import pandas as pd

# #导入绘图工具
# import matplotlib.pyplot as plt
# #一般作为机器学习绘图用, 但是也能绘出更cool的图
# import seaborn as sns

#mysql 数据源工具
import sqlalchemy as db

#系统及命令行
import sys

import json
import requests

In [41]:
arguments=sys.argv
if len(arguments) <= 4:
    print("usage: {} db_host db_user db_password esHost. For example, {} 10.10.0.215:3306 root 82f12dbf497b 10.10.0.223:9200".format(arguments[0], arguments[0]))
    sys.exit(-1)

profile_db = "10.10.0.215:3306"
username = "root"
password = "82f12dbf497b"
esHost="10.10.0.223:9200"
esIndex="profile_service"
esType="extra_profile"

profile_db = sys.argv[1]
username = sys.argv[2]
password = sys.argv[3]
esHost = sys.argv[4]

print("database: {}, username: {}, password: {}, elasticsearch: {}".format(profile_db, username, password, esHost))

mysql_conn_str = 'mysql+pymysql://{}:{}@{}/profile_service?charset=utf8mb4'.format(username, password, profile_db)

user_profile_query = '''select login_id,device_type,device_id,device_address,email,image,
                    nick_name,social_id,sex,phone_no,school_name,major,sign_ature,weichat_image,
                    weichat_nick,openid from user_profile where login_id is not null'''

In [42]:
try:
    engine = db.create_engine(mysql_conn_str)
    connection = engine.connect()
    df = pd.read_sql_query(user_profile_query, connection)
except:
    print("读取数据错误")
    raise "读取数据错误"
connection.close()

In [47]:
def getJsonDataFromRow(row):
    loginId = row[0]
    data = {"loginId": loginId}
    if row[1] != None:
        data["deviceType"] = row[1]
    if row[2] != None:
        data["deviceId"] = row[2]
    if row[3] != None:
        data["deviceAddress"] = row[3]
    if row[4] != None:
        data["email"] = row[4]
    if row[5] != None:
        data["image"] = row[5]
    if row[6] != None:
        data["nickName"] = row[6]
    if row[7] != None:
        data["socialId"] = row[7]
    if row[8] != None:
        data["sex"] = row[8]
    if row[9] != None:
        data["phoneNo"] = row[9]
    else:
        data["phoneNo"] = loginId
    if row[10] != None:
        data["schoolName"] = row[10]
    if row[11] != None:
        data["major"] = row[11]
    if row[12] != None:
        data["signAture"] = row[12]
    if row[13] != None:
        data["weichatImage"] = row[13]
    if row[14] != None:
        data["weichatNick"] = row[14]
    if row[15] != None:
        data["openid"] = row[15]
    updateData = {"doc_as_upsert":True, "doc": data}
    return json.dumps(updateData)

In [54]:
for row in df.values:
    loginId = row[0]
    postUrl = "http://{}/{}/{}/{}/_update".format(esHost, esIndex, esType, loginId)
    rowData = getJsonDataFromRow(row)
    requests.post(url = postUrl, data = rowData.encode('utf-8'), headers = {"Content-Type":"application/json"})

{"doc_as_upsert": true, "doc": {"loginId": "t1", "deviceType": "test6", "deviceId": "test6", "deviceAddress": "test6", "email": "test6", "image": "test6", "nickName": "test6", "socialId": "test6", "phoneNo": "t1"}}
{"doc_as_upsert": true, "doc": {"loginId": "t2", "deviceType": "test6", "deviceId": "test6", "deviceAddress": "test6", "email": "test6", "image": "test6", "nickName": "test6", "socialId": "test6", "phoneNo": "t2"}}
{"doc_as_upsert": true, "doc": {"loginId": "t3", "deviceType": "test77", "deviceId": "test77", "deviceAddress": "test77", "email": "test77", "image": "test77", "nickName": "test77", "socialId": "test77", "phoneNo": "t3"}}
{"doc_as_upsert": true, "doc": {"loginId": "test77", "deviceType": "test77", "deviceId": "test77", "deviceAddress": "test77", "email": "test77", "image": "the8979", "nickName": "test77", "socialId": "test77", "sex": "test77", "phoneNo": "test77", "schoolName": "test123", "major": "test77", "signAture": "test77"}}
{"doc_as_upsert": true, "doc": {"